<a href="https://colab.research.google.com/github/elichen/karpathyGPT/blob/main/Let's_build_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -q https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [2]:
text = open("input.txt").read()

In [3]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
vocab_size, "".join(chars)

(65, "\n !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz")

In [4]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])

In [5]:
import torch
data = torch.tensor(encode(text))

In [6]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [7]:
block_size = 8
batch_size = 4

def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x, y

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BigramModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs)
        return embeds

    def calculate_loss(self, inputs, targets):
        logits = self(inputs)
        return F.cross_entropy(logits.view(-1, vocab_size), targets.view(-1))

    def generate(self, input_batch, max_new_tokens):
        generated = input_batch
        for _ in range(max_new_tokens):
            logits = self(generated)
            last_logits = logits[:, -1, :]
            probabilities = F.softmax(last_logits, dim=-1)
            next_tokens = torch.multinomial(probabilities, 1).squeeze(1)
            generated = torch.cat((generated, next_tokens.unsqueeze(1)), dim=1)
        return generated

model = BigramModel(vocab_size)

xb,yb = get_batch("train")
print(model(xb).shape)
out = model.calculate_loss(xb,yb)
out

torch.Size([4, 8, 65])


tensor(4.6148, grad_fn=<NllLossBackward0>)

In [9]:
x = torch.zeros((1,1), dtype=torch.long)
print(decode(model.generate(x, max_new_tokens=100)[0].tolist()))


Ech;vU:Yotv3-lHh:YyC'BYj?N?QudPDx-P'OyGPd;GwP'kxTnLYo!j:dehV3sixje QClXsr:az.Hp-CTZ-r xkLaz$KAhxUheM


In [10]:
import torch.optim as optim

optimizer = optim.AdamW(model.parameters(), lr=1e-3)

In [11]:
batch_size = 32
for steps in range(10000):
  xb,yb = get_batch("train")
  loss = model.calculate_loss(xb,yb)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
print(loss.item())

2.4959936141967773


In [12]:
print(decode(model.generate(x, max_new_tokens=400)[0].tolist()))


Youtt te ke arartin s; ld pr, tedf gheron thilitreyssmarutr
Yoptrmube.
HE itho f!
SHainales hovest, were!
ge S:
HICENouckeabe hed me h ses at t,
LI lorir:

O:
ADes bre h s, lastonout trit whe?
HA:
xe ce o roures mowel. fama at! then merompt monttolatrtot.


NAnrve I
Ay d, stVIf it:
TI t fowemin,
KI ld shy.
Win, therioniouno tait ficheso fony ts y's OHim r n s,'
Tor d, e laur cal thozntus, ght, s y
